### Local functions 

local functions are defined inside other functions and are created each time the enclosing function is called.

1. local functions are subject to the same LEGB rule as regular functions, local -> enclosing -> global -> built-in
2. local functions are not members of enclosing functions
3. local functions are used for specialized, one-off functions
4. they aid in code organization and readability
5. they are similar to lambda, but more general

In [2]:
store = []

def sort_by_last_letter(strings):
    def last_letter(string): 
        return string[-1]
    
    store.append(last_letter)
    print(last_letter)
    return sorted(strings, key=last_letter)

In [3]:
sort_by_last_letter(['abc', 'def', 'ghi'])

<function sort_by_last_letter.<locals>.last_letter at 0x000001D2F9081A68>


['abc', 'def', 'ghi']

In [4]:
sort_by_last_letter(['abc', 'def', 'ghi'])

<function sort_by_last_letter.<locals>.last_letter at 0x000001D2F9081948>


['abc', 'def', 'ghi']

In [5]:
sort_by_last_letter(['abc', 'def', 'ghi'])

<function sort_by_last_letter.<locals>.last_letter at 0x000001D2F9081798>


['abc', 'def', 'ghi']

In [6]:
g = 'global' # global

def outer(p='param'):
    l = 'local' # enclosing
    def inner(): # local
        print (g, p, l)
    
    inner()
    
outer()

global param local


### Closure

1. Once a local function is returned from its enclosing scope, that enclosing scope is gone, along with any local object it defined. How can a local function operation without that enclosing scope? The answer is local function forms what is known as a closure. 

2. A closure maintains reference to objects from earlier scopes.

3. Python implements closure with a special attribute named __closure__. If a function closes over any objects, the next function has a __closure__ attribute, which maintains necassary reference to these objects.

4. A common use for closure is function factories, which are functions that return new, specialized functions. In other words, the function factory takes some arguments, it then creates a local function, which takes its own argument but also uses the arguments passed to the factory. The combination of runtime function definition enclosures makes this possible. 

5. LEGB rule does not apply when we make new name bindings. 

In [7]:
# functions can be passed to or returned from another function
# functions can be treated like any other object
def enclosing():
    def local_func():
        print('local func')
    return local_func

f = enclosing()
f()

local func


In [8]:
def outer():
    x = 3
    def inner(y):
        return x + y
    return inner

f = outer()
f(4)

7

In [11]:
def enclosing():
    x = 'closed over'
    def local():
        print (x)
    return local

l = enclosing()
l()

l.__closure__ # l is a closure, which is referring to a single object, which is x in this case.

closed over


(<cell at 0x000001D2F90A25B8: str object at 0x000001D2F90BC130>,)

In [13]:
def raise_to(exp): # function factories
    def raise_to_exp(x):
        return pow(x, exp)
    return raise_to_exp

square = raise_to(2)
cube = raise_to(3)

print(square(2))
print(cube(2))

4
8


In [14]:
message = 'global'

def enclosing():
    message = 'enclosing'
    def local():
        message = 'local'
        print(message)
        
    print('enclosing message', message)
    local()
    print('enclosing message', message)
    
print('glocal message', message)
enclosing()
print('glocal message', message)

glocal message global
enclosing message enclosing
local
enclosing message enclosing
glocal message global


In [16]:
message = 'global'

def enclosing():
    message = 'enclosing'
    def local():
        global message # glocal keyword can introduce glocal namespace into local namespace. 
        message = 'local'
        print(message)
        
    print('enclosing message', message)
    local()
    print('enclosing message', message)
    
print('glocal message', message)
enclosing()
print('glocal message', message)

glocal message global
enclosing message enclosing
local
enclosing message enclosing
glocal message local


In [17]:
message = 'global'

def enclosing():
    message = 'enclosing'
    def local():
        nonlocal message # nonlocal keyword can introduce names from enclosing namespace into local namespace. 
        message = 'local'
        print(message)
        
    print('enclosing message', message)
    local()
    print('enclosing message', message)
    
print('glocal message', message)
enclosing()
print('glocal message', message)

glocal message global
enclosing message enclosing
local
enclosing message local
glocal message global


In [19]:
import time

def time_maker():
    last_called = None
    
    def elapsed():
        nonlocal last_called
        now = time.time()
        if not last_called:
            last_called = now
            return None
        result = now - last_called
        last_called = now
        return result
    
    return elapsed

t = time_maker()

print(t())
print(t())
print(t())

None
0.0
0.0


### Decorator

1. Decorators can modify or enhance functions without changing their definitions. In python, decorators are callable objects that take in a callable and returns a callable.

2. Classes and instances can be used as decorators.

3. Multiple decorators are processed in reverse order.

4. Naive decorators can lose important metadata. Functools.wraps() properly updates metadata on wrapped functions. Functools.wraps is a function decorator which you apply to your wrapper functions. The decorator takes the function to be decorated as an argument and it does the hard work of updating the wrapper function with the wrapped function's attributes.

5. Decorators can enhance maintainability, readability and scalability of designs.

In [22]:
def escape_unicode(f): # f is a function to be decorated
    def wrap(*args, **kwargs): # *args and **kwargs are used to accpet any number of parameters
        x = f(*args, **kwargs)
        return ascii(x) # escape non-ascii characters
    return wrap

print(northern_city())

@escape_unicode
def northern_city():
    return 'newyork'

northern_city()

'newyork'


"'newyork'"

In [23]:
# class as decorators
class CallCount:
    def __init__(self, f):
        self.f = f
        self.count = 0
        
    def __call__(self, *args, **kwargs): # instance must be callable and must supports __call__ method
        self.count += 1
        return self.f(*args, **kwargs)
           
@CallCount
def hello(name): # applying class decorator creates a new instance
    print('Hello, {}'.format(name))

hello('Betty')
hello('Ling')
hello('Molly')

hello.count

Hello, Betty
Hello, Ling
Hello, Molly


3

In [24]:
# instance as decorators
class Trace:
    def __init__(self):
        self.enabled = True
        
    def __call__(self, f): # the return value of call must be callables
        def wrap(*args, **kwargs):
            if self.enabled:
                print('Calling {}'.format(f))
            return f(*args, **kwargs)
            
        return wrap

tracer = Trace()

@tracer
def rotate_string(string): # decorating with instance calls an instance
    return string[1:] + [string[0]]

l = [1, 2, 3]
l = rotate_string(l)
l = rotate_string(l)
l

Calling <function rotate_string at 0x000001D2F916C0D8>
Calling <function rotate_string at 0x000001D2F916C0D8>


[3, 1, 2]

In [25]:
tracer.enabled = False
l = rotate_string(l)
l = rotate_string(l)
l

[2, 3, 1]

In [27]:
def escape_unicode(f): # f is a function to be decorated
    def wrap(*args, **kwargs): # *args and **kwargs are used to accpet any number of parameters
        x = f(*args, **kwargs)
        return ascii(x) # escape non-ascii characters
    return wrap

class Trace:
    def __init__(self):
        self.enabled = True
        
    def __call__(self, f): # the return value of call must be callables
        def wrap(*args, **kwargs):
            if self.enabled:
                print('Calling {}'.format(f))
            return f(*args, **kwargs)
            
        return wrap

tracer = Trace()

@tracer
@escape_unicode
def island_maker(name): # multiple decorators
    return name + '&y'

island_maker('python')

Calling <function escape_unicode.<locals>.wrap at 0x000001D2F916CC18>


"'python&y'"

In [28]:
tracer.enabled = False
island_maker('anaconda')

"'anaconda&y'"

In [30]:
class IslandMaker:
    def __init__(self, suffix):
        self.suffix = suffix
        
    @tracer
    def make_island(self, name): # decorator on methods
        return name + self.suffix
    
im = IslandMaker(' Island')
tracer.enabled = True
im.make_island('python')

Calling <function IslandMaker.make_island at 0x000001D2F9159048>


'python Island'

In [32]:
def hello():
    '''print a well-known messgae'''
    print('hello world')
    
print(hello.__name__)
print(hello.__doc__)

help(hello)

hello
print a well-known messgae
Help on function hello in module __main__:

hello()
    print a well-known messgae



In [33]:
def noop(f):
    def noop_wrapper():
        return f()
    return noop_wrapper

@noop
def hello():
    '''print a well-known messgae'''
    print('hello world')
    
print(hello.__name__)
print(hello.__doc__)

help(hello)

noop_wrapper
None
Help on function noop_wrapper in module __main__:

noop_wrapper()



In [35]:
def noop(f):
    def noop_wrapper():
        return f()
    
    # a bit ugly
    noop_wrapper.__name__ = f.__name__
    noop_wrapper.__doc__ = f.__doc__
    
    return noop_wrapper

@noop
def hello():
    '''print a well-known messgae'''
    print('hello world')
    
print(hello.__name__)
print(hello.__doc__)

help(hello)

hello
print a well-known messgae
Help on function hello in module __main__:

hello()
    print a well-known messgae



In [36]:
import functools

def noop(f):
    @functools.wraps(f)
    def noop_wrapper():
        return f()
    return noop_wrapper

@noop
def hello():
    '''print a well-known messgae'''
    print('hello world')
    
print(hello.__name__)
print(hello.__doc__)

help(hello)

hello
print a well-known messgae
Help on function hello in module __main__:

hello()
    print a well-known messgae



In [38]:
def check_non_negative(index): # not a decorator
    def validator(f): # decorator that takes callable as an argument and returns callables
        def wrapper(*args):
            if args[index] < 0:
                raise ValueError('Argument {} must be non-negative!'.format(index))
            return f(*args)
        return wrapper
    return validator

@check_non_negative(1)
def create_list(val, size):
    return [val] * size

create_list(3, -2)

ValueError: Argument 1 must be non-negative!